In [1]:
#YC95
#HO3


6.1 Predicting Boston Housing Prices. The file BostonHousing.csv contains information
collected by the US Bureau of the Census concerning housing in the area of
Boston, Massachusetts. The dataset includes information on 506 census housing tracts
in the Boston area. The goal is to predict the median house price in new tracts based
on information such as crime rate, pollution, and number of rooms. The dataset contains
13 predictors, and the outcome variable is the median house price (MEDV).

Table 6.11 describes each of the predictors and the outcome variable.


a. Why should the data be partitioned into training and validation sets? What will the

training set be used for? What will the validation set be used for?
b. Fit a multiple linear regression model to the median house price (MEDV) as a
function of CRIM, CHAS, and RM. Write the equation for predicting the median
house price from the predictors in the model.

c. Using the estimated regression model, what median house price is predicted for a
tract in the Boston area that does not bound the Charles River, has a crime rate of
0.1, and where the average number of rooms per house is 6?

In [2]:
pip install dmba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 40.6 MB/s eta 0:00:00


In [3]:
import os
import calendar
from pathlib import Path
import numpy as np
import networkx as nx
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.plotting import scatter_matrix, parallel_coordinates
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pylab as plt
import dmba

%matplotlib inline

Colab environment detected.


In [4]:
# Read the CSV file into a DataFrame BostonHousing.csv
bostonHousing_df = dmba.load_data('BostonHousing.csv')
bostonHousing_df = bostonHousing_df.rename(columns={'CAT. MEDV': 'CAT_MEDV'})
bostonHousing_df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV,CAT_MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,1
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,1
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,1


In [5]:
# Data set different way
bostonHousing_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV,CAT_MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806,0.166008
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104,0.372456
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000,0.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000,0.000000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000,0.000000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000,0.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000,1.000000


In [6]:
# a. Why should the data be partitioned into training and validation sets? What will the training set be used for?
#  What will the validation set be used for?

Data partitioning into training and validation sets is a fundamental practice in machine learning and predictive modeling. The training set is employed to construct the model, uncovering relationships between predictor variables and the target variable. This process enables the model to learn patterns and make predictions. On the other hand, the validation set serves as an independent dataset used to assess the model's predictive accuracy.

By applying the trained model to the validation data, one can evaluate how well it generalizes to new, unseen data. Error metrics are calculated during validation, providing insights into the model's performance and its ability to make accurate predictions.

In summary, partitioning the data into training and validation sets is essential for developing and evaluating predictive models for Boston housing prices. The training set is used for model learning and development, while the validation set is used to assess the model's generalization ability, evaluate its performance. This practice ensures that the model can provide reliable predictions on new, unseen housing tracts in Boston based on the provided predictors.

In [7]:
# Values of Counts
bostonHousing_df.CRIM.value_counts()
bostonHousing_df.CHAS.value_counts()
bostonHousing_df.RM.value_counts()

5.713    3
6.167    3
6.127    3
6.229    3
6.405    3
        ..
5.859    1
6.416    1
5.572    1
5.880    1
6.976    1
Name: RM, Length: 446, dtype: int64

 B) Fit a multiple linear regression model to the median house price (MEDV) as a function of CRIM, CHAS, and RM. Write the equation for predicting the median house price from the predictors in the model.

In [8]:
# Define predictors (features) and target variable
X = bostonHousing_df[['CRIM', 'CHAS', 'RM']]
y = bostonHousing_df['MEDV']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train, y_train are used to train the model
# X_val, y_val are used to validate the model

In [9]:
# Fit the multiple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Print the coefficients (equation for predicting MEDV)
print('Intercept:', model.intercept_)
print('Coefficients:', model.coef_)

Intercept: -30.521139801711087
Coefficients: [-0.25780835  3.87897489  8.54506433]


C) Using the estimated regression model, what median house price is predicted for a tract in the Boston area that does not bound the Charles River, has a crime rate of
0.1, and where the average number of rooms per house is 6?

In [10]:
# Predict median house price for a specific tract
# Tract doesn't bound the Charles River, crime rate = 0.1, and average rooms = 6
new_tract = [[0.1, 0, 6]]
predicted_price = model.predict(new_tract)
print('Predicted Median Boston House Price:', predicted_price)

Predicted Median Boston House Price: [20.72346535]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [11]:
# HO3